# SWE Agent Workshop: Build an Autonomous Developer

## What You'll Build (35-40 minutes)

A **fully autonomous software agent** that:
- Automatically handles GitHub issues
- Creates pull requests with complete solutions
- Integrates with Slack for team collaboration
- Runs background coding tasks using Claude Code
- Accesses knowledge bases for expert-level responses

---

# Part 1: Initial Setup (8 minutes)

## Goal: Create and test your SWE agent with Claude Code integration

**Expected Output**: Working agent that can coordinate with Claude Code for development tasks

### How to do it:

1. **Go to template**: https://app.xpander.ai/templates/d5175e28-6608-484b-9d4f-69ce14e699b0
2. **Login with GitHub** (watch video): https://assets.xpanderai.io/vidoes/workshops/august-2025/01-github-connect.mp4
3. **Test your agent** with: *"Ask Claude to write hello world in different languages"*

<div align="center">
<img src="images/first-prompt.png" style="max-height: 600px; width: auto;">
</div>

### Step completion check:
-  Agent responds with code in multiple languages
-  Function logs show your agent coordinating with Claude Code
-  You understand the two modes: Inline (direct code) and Background (GitHub PRs)

**Key insight**: Your agent can delegate unlimited development work to Claude Code. This is multi-agent collaboration that scales infinitely.

# Part 2: Local Configuration & Custom Capabilities (15 minutes)

## Goal: Connect locally, add custom tools, and enable knowledge-based responses

**Expected Output**: Agent with persistent memory, GitHub tools, and expert-level documentation access

In [ ]:
# Install dependencies
!pip install "xpander-sdk[agno]" openai requests pypdf pgvector -q
!npm install -g xpander-cli

In [ ]:
!xpander login

# Get your agent details - copy the Agent ID and API Key from output
!xpander agent get "SWE Agent"

In [ ]:
# Run the agent via CLI
!xpander agent invoke "SWE Agent" "Can you write bubble sort algorithm in Python?"

## Interact with the agent using SDK

In [ ]:
import os
from dotenv import load_dotenv
from xpander_sdk import Backend
from agno.agent import Agent

load_dotenv()

# Update these with your values from above
os.environ['XPANDER_AGENT_ID'] = 'your-agent-id-here'
os.environ['XPANDER_API_KEY'] = 'your-api-key-here' 

backend = Backend()
swe_agent = Agent(**backend.get_args())
print("✅ SWE Agent initialized!")

In [ ]:
# Inspect available capabilities
print("🔧 Available Tools:")
for tool in swe_agent.tools:
    print(f"   • {tool.__name__}")
    
print(f"\n💾 Database: {swe_agent.storage.db_url}")
print(f"📋 Instructions: {len(swe_agent.instructions)} rules loaded")

## Test Session Memory

**Goal**: Verify persistent memory across interactions

**Key differentiator**: PostgreSQL storage vs ChatGPT's forgetting conversations

In [ ]:
# Test 1: Ask for code
response = swe_agent.run("Write a Python function to validate email addresses")
print("Response preview:", response.content[:200] + "...")
print("Session ID:", response.session_id)

In [ ]:
# Test 2: Verify memory
response = swe_agent.run("What did I just ask you to do?")
print("Memory test result:", response.content[:150] + "...")

In [ ]:
swe_agent.run("What did I ask you?", session_id="new-session-id-123")
print("Memory test 2 result:", response.content[:150] + "...")

## Add Custom Tool - GitHub Operations

**Goal**: Extend agent capabilities with GitHub API integration

**Expected Output**: Agent can star repositories on command

In [ ]:
from xpander_sdk import register_tool
import requests

os.environ['GITHUB_TOKEN'] = 'your-github-token-here' # get it from https://github.com/settings/tokens . Create "Classic" token

@register_tool
def star_github_repo(repo_full_name: str) -> dict:
    """Star a GitHub repository
    
    Args:
        repo_full_name: Repository in format 'owner/repo'
    """
    token = os.environ.get("GITHUB_TOKEN")
    if not token:
        return {"error": "No GitHub token configured"}
    
    response = requests.put(
        f"https://api.github.com/user/starred/{repo_full_name}",
        headers={"Authorization": f"token {token}"}
    )
    return {"success": response.status_code == 204, "repo": repo_full_name}

# Reload agent to pick up new tool
swe_agent = Agent(**backend.get_args())
print("✅ GitHub tool registered and agent reloaded")

In [ ]:
# Test the custom tool
response = swe_agent.run("Please star the xpander-ai/xpander.ai repository")
print("Tool test result:", response.content[:120] + "...")

## Add Knowledge Base - Expert Documentation

**Goal**: Give agent access to authoritative sources for expert-level responses

**Expected Output**: Agent can answer questions using official documentation

In [ ]:
import requests
from agno.document.chunking.fixed import FixedSizeChunking
from agno.knowledge.pdf import PDFKnowledgeBase
from agno.vectordb.pgvector import PgVector

def download_pdf(url, filename):
    """Download PDF from URL to local file"""
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        f.write(response.content)
    return filename

# URLs to download - updated with correct paper
pdf_urls = [
    "https://githubtraining.github.io/training-manual/legacy-manual.pdf"
]

# Download PDFs and create PDFConfig list
pdf_configs = []
for i, url in enumerate(pdf_urls):
    filename = f"document_{i+1}.pdf"
    try:
        pdf_path = download_pdf(url, filename)
        pdf_configs.append({"path": pdf_path})
        print(f"Downloaded: {pdf_path}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")

# Create knowledge base from all downloaded PDFs
knowledge_base = PDFKnowledgeBase(
    path=pdf_configs,  # List of PDFConfig dictionaries
    vector_db=PgVector(
        table_name="swe_documentation",
        db_url=swe_agent.storage.db_url,
        auto_upgrade_schema=True,
        schema=swe_agent.storage.schema
    ),
    chunking_strategy=FixedSizeChunking(chunk_size=1000)
)

# Load documents
knowledge_base.load(recreate=False)

# Enable knowledge base for the agent
swe_agent.knowledge = knowledge_base
swe_agent.search_knowledge = True

print(f"✅ Knowledge base loaded with {len(pdf_configs)} PDFs!")

In [ ]:
# Test knowledge-enhanced responses
response = swe_agent.run("What's the official way to do git cherry-pick according to GitHub?")
print("Knowledge-based answer:", response.content[:250] + "...")

### Part 2 Completion Check:
- Agent has persistent memory across sessions
- Custom GitHub tool works correctly
- Knowledge base provides authoritative answers
- Multi-agent system coordinates Claude Code for complex tasks

# Part 3: Slack Integration (8 minutes)

## Goal: Connect agent to team workflows via Slack

**Expected Output**: Agent responds to team requests in Slack channels

### How to do it:

1. **Watch setup video**: https://assets.xpanderai.io/vidoes/workshops/august-2025/02-slack.mp4
2. **Join Xpander Community**: https://bit.ly/xpander-slack (use browser)
3. **Create channel**: `swe-agent-tasks-yourname`
4. **Configure Smart Engage**:
   - Agent Settings → Smart Engage
   - Enable Slack integration
   - Connect workspace (tokens from https://api.slack.com/reference/manifests#config-tokens)
   - Select your channel

<div align="center">
<img src="images/slack-1.png" style="max-height: 500px; width: auto;">
</div>


1. **Test**: Send message asking coding related question. It should work without @ (hashtag) !

<div align="center">
<img src="images/slack-result.png" style="max-height: 500px; width: auto;">
</div>


### Step completion check:
-  Agent responds to Slack messages
-  Team members can request development help
-  Agent can execute background tasks from Slack requests

**Key insight**: Your agent is now a collaborative team member that never sleeps and has unlimited development capacity.

# Part 4: GitHub Webhook Automation (12 minutes)

## Goal: Complete autonomous pipeline for GitHub issue processing

**Expected Output**: Agent automatically handles new GitHub issues end-to-end

**Why webhook filtering matters**: GitHub sends webhooks on every issue event (opened, closed, commented, etc.). We only want "opened" issues, and we want to transform the raw JSON payload into AI-ready format instead of overwhelming the agent with unnecessary data.

## Step 1: Create repo & Configure GitHub Webhook

**Task**: Set up webhook to send issue events to your agent

### How to do it:
1. Create new repository in GitHub and commit something to it
2. Go to `https://github.com/<your-org>/<your-repo>/settings/hooks`
3. Add webhook with this URL: (You can also find it in the Agent Task Sources)
```
https://webhook.xpander.ai?agent_id=<your-agent-id>&x-api-key=<your-api-key>&asynchronous=true
```

<div align="center">
<img src="images/github-webhook.png" style="max-height: 500px; width: auto;">
</div>

3. Select "Issues" events only
4. Verify in activity view that webhook events arrive

<div align="center">
<img src="images/webhook-xpander.png" style="max-height: 500px; width: auto;">
</div>


## Step 2: Add GitHub Issues Tool

**Task**: Enable agent to comment on and manage GitHub issues

### How to do it:
1. Xpander platform → **Connectors**
2. Add **GitHub Issues** connector and authenticate
3. **Add to your agent**

<div align="center">
<img src="images/github-issues.png" style="max-height: 500px; width: auto; margin-right: 20px;">
<img src="images/github-issues-2.png" style="max-height: 500px; width: auto;">
</div>

4. Click save and deploy. 

## Step 3: Configure Collaborative Instructions

**Task**: Make agent communicate transparently during issue resolution

### How to do it:
Add this instruction to your agent:

`When receiving an issue from GitHub, you must begin by replying to the issue with your plan. Do not start coding before commenting on the issue. Once you have commented, proceed to solve the issue by writing code and creating a pull request. Finally, comment on your workflow in the issue, even if you encountered problems.`

**Save and Deploy**

Test it. Create issue on github, or use the chat and ask the agent to comment on some github issue. 

## Step 4: Advanced Webhook Filtering (Bonus)

**Goal**: Transform raw GitHub webhooks into AI-ready format

**Expected Output**: Agent receives clean, filtered issue data instead of raw JSON

### Why this matters:
- GitHub sends webhooks for every issue action (opened, closed, edited, etc.)
- Raw webhook payloads contain 100+ fields your agent doesn't need
- We want to filter for "opened" issues only and extract just the essential data

### How to do it:

In [ ]:
# Download your agent's code for customization
!xpander agent init "SWE Agent" --framework agno --folder ./swe-agent

In [ ]:
# Check downloaded files
!ls ./swe-agent/

### Add webhook filtering to `xpander_handler.py`:

**Add this function at the top:**

In [ ]:
# This is the filtering function to add to xpander_handler.py
webhook_filter_code = '''
import json

def process_github_event(event):
    """Filter GitHub webhooks to only process opened issues
    
    Returns AI-ready format: issue body + URL
    Returns False for events we want to ignore
    """
    if isinstance(event, str):
        event = json.loads(event)
    
    # Only process newly opened issues
    if event.get("action") == "opened" and "issue" in event:
        issue_body = event['issue'].get('body', 'No description')
        issue_url = event['issue'].get('html_url')
        return f"{issue_body} | {issue_url}"
    
    return False  # Ignore all other events
'''

print("Copy this code to the top of ./swe-agent/xpander_handler.py")
print(webhook_filter_code)

**Add this filter logic in the main handler function:**

In [ ]:
# This is the integration code to add in xpander_handler.py
integration_code = '''
# Add this after: agno_agent = Agent(**agno_args)

# Filter GitHub webhook events
github_payload = process_github_event(task.input.text)
if github_payload is not False:
    task.input.text = github_payload
    print(f"Processed GitHub issue: {github_payload[:100]}...")
elif "issue" in task.input.text.lower():
    # If it looks like a GitHub event but we filtered it out
    print("Ignored GitHub event (not a new issue)")
    return  # Don't process this event

# Continue with: await agno_agent.arun(...)
'''

print("Add this code in xpander_handler.py before the agno_agent.arun call:")
print(integration_code)

### Test locally:

# Run this in terminal after editing xpander_handler.py:

In [ ]:
!cd swe-agent && python3 -m venv .venv && source .venv/bin/activate && pip install -r requirements.txt


In [ ]:
!cd swe-agent && xpander agent dev

## While running, re-deliver a GitHub webhook to test filtering

<div align="center">
<img src="images/github-retrigger.png" style="max-height: 500px; width: auto;">
</div>

Deploy to production: `xpander deploy` (Optional)

In [ ]:
!cd swe-agent && xpander deploy --yes

### Part 4 Completion Check:
-  GitHub webhook configured for Issues events
-  Agent can comment on GitHub issues
-  Agent follows collaborative workflow (comment → code → PR → update)
-  (Bonus) Webhook filtering processes only relevant events

**Test the complete pipeline**: Create a GitHub issue and watch your autonomous development pipeline in action!

# Workshop Complete! 🎉

## What You've Built:
✅ **Multi-agent system** with Claude Code integration  
✅ **Custom tools** and persistent memory  
✅ **Knowledge base** for expert responses  
✅ **Slack collaboration** for team integration  
✅ **GitHub automation** with intelligent webhook filtering  

## Your Autonomous Development Pipeline:
```
GitHub Issue → Filtered Webhook → Agent Analysis → 
Issue Comment → Code Development → PR Creation → Team Notification
```

## Next Steps:
- Scale across multiple repositories
- Add specialized tools for your workflows  
- Integrate with project management tools
- Build domain-specific knowledge bases

**You now have a 24/7 AI developer that can handle the complete software development lifecycle.**